# Exploring Residential House Prices in Singapore
By: Chengyi (Jeff) Chen

In this project, we will first gather a dataset of Singapore Residential housing prices and their features from [99.co](https://www.99.co/singapore), a popular online marketplace for property in Singapore by building a simple ETL pipeline. Afterwards, we will explore the data we've scraped using the CRISP-DM process. This notebook will cover the web-scraping, data extraction, and transformation part 1 of the ETL pipeline.

---
## Table of Contents

1. [Web Scraping](#web-scrape)
2. [Extract](#extract)
3. [Transform Part 1](#transform)

In [1]:
# General Utilities for Web Scraping
import sys
import os
from os import listdir
from os.path import isfile, join, isdir
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import date, datetime, timedelta
from dateutil import relativedelta
import csv
import json
from collections import defaultdict
from collections import Counter
from imp import reload

# Machine Learning Utitilies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Url
base_url = 'https://www.99.co/singapore/sale'

<a id='web-scrape'></a>

---
# Web Scraping for Data

We'll first scrape the 99.co website for all the data we can find by first retrieving all the links for the "for-sale" properties right now, and also information specific to each house like what amenities are currently available, it's proximity to the nearest MRT (Mass Rapid Transport / train) station, etc. We will store them in various formats like csv and json until we've decided what questions we want to figure out using this data in the business understanding section of CRISP-DM.

### Helper Functions for Web Scraping

In [2]:
import webscraper_99co
from webscraper_99co import *

reload(webscraper_99co);

### Get URLs for each "for-sale" property on 99.co

In [3]:
property_links = get_property_links(base_url)
len(property_links)

100%|██████████| 400/400 [06:58<00:00,  1.53it/s]


10000

In [4]:
# Save these property links to csv so that we 
# don't have to scrape them everytime
property_links_df = pd.DataFrame(property_links, columns=['url'])
property_links_df.head()

,url
0,https://www.99.co/singapore/sale/property/563-...
1,https://www.99.co/singapore/sale/property/macp...
2,https://www.99.co/singapore/sale/property/puay...
3,https://www.99.co/singapore/sale/property/hill...
4,https://www.99.co/singapore/sale/property/the-...


In [5]:
property_links_df.to_csv('./data/99.co/property_links.csv')

In [6]:
# Read in the property links csv
property_df = pd.read_csv('./data/99.co/property_links.csv', index_col=[0])
property_df.head()

,url
0,https://www.99.co/singapore/sale/property/563-...
1,https://www.99.co/singapore/sale/property/macp...
2,https://www.99.co/singapore/sale/property/puay...
3,https://www.99.co/singapore/sale/property/hill...
4,https://www.99.co/singapore/sale/property/the-...


### Get All the information available on 99.co for each property

In [7]:
# Get all the property information for the day
# This will take really long so I recommend running it in a tmux session 
# OR
# Take away the API calls for the transaction history and pricing trends
# because we can just call them for a subset of the properties later in the
# project like what I did here
get_all_property_listing_data(property_df)

  0%|          | 37/10000 [01:13<7:36:25,  2.75s/it] 

cluster_id for amber-park-condo-YVFX8gDSn6fjqMV3qtfEH5 cannot be found.


  2%|▏         | 189/10000 [07:37<9:01:01,  3.31s/it] 

cluster_id for marine-blue-condo-a7Y9C4BYJ9penwNW8U52ph cannot be found.


  4%|▎         | 361/10000 [16:16<10:04:26,  3.76s/it]

...

(./data/99.co/data/2019_07_11/tribeca-condo-HdbvGwNwumH2w2XmaYhK9J) already exists


  9%|▊         | 867/10000 [38:49<6:13:59,  2.46s/it]

(./data/99.co/data/2019_07_11/cityvista-residences-condo-HL8oamq64RNBnV5srSBgsc) already exists


  9%|▉         | 896/10000 [40:36<11:28:10,  4.54s/it]

SSLError: HTTPSConnectionPool(host='www.99.co', port=443): Max retries exceeded with url: /api/v1/web/clusters/deMaLuQzGMonwbSYZfbPxgQf/nearby (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')")))

In [ ]:
# Take a look at how much data we got
properties_dir = './data/99.co/data/{}/'.format((date.today() - timedelta(days=1)).strftime("%Y_%m_%d"))
properties = [join(properties_dir, property_dir) for property_dir in listdir(properties_dir) if isdir(join(properties_dir, property_dir))]
len(properties)

<a id='extract'></a>

---
# Extract

Let's take a look at the current json data that we have and see which we can include / exclude from the final dataset.

In [ ]:
# Let's see the commute and nearby data 
# of a random property in our list
with open(join(properties[0], 'commute_and_nearby_data.json'), 'r') as f:
    commute_and_nearby_data = json.load(f)
    
commute_and_nearby_data.keys()

## Schools

In [ ]:
commute_and_nearby_data['school']

- As observed, we have information about how much it'll be to get to a few of the nearest schools around the property by different modes of transportation. 
- At first glance, it does seem pretty tough as to which pieces of information we should include in building our models. 
    - Example 1 - Having a school close by might not necessarily have a positive effect on the price of a house depending of the reputation of the school as well. Schools that are very prestigious around the house might drive up the price of the property, but we don't have this semantic information available. 
    - Example 2 - Schools that are not as accessible by private transportation might be highly accessible by public transport because of how well developed Singapore's public transportation system is and therefore it might be better to include information about each mode of transport rather than using one specific type as a proxy for all. 
- Hence, as a temporary simplification, we'll just use the breakdown of the number of each type of school nearby (primary, secondary, junior college, university, others) to include into our final dataset. For properties that do not have any schools nearby, we'll just impute 0 for each school category.

## Subway Station (E.g. MRT - Mass Rapid Transport)

In [ ]:
commute_and_nearby_data['subway_station']

- Similarly, we have information about the cost of transportation modes to get to the subway station. In contrast to schools, it does not matter as much about which subway station is nearby, but rather if there are subway stations nearby at all. So we'll only really need to care about the number of subway stations nearby, which is what we will include in our final dataset.
- Furthermore, at least from my own personal experience, it is highly unlikely that one will drive to and from a subway station. Hence, in terms of numerical data, we can also include the average `duration` for transit (how long on average it'll take by bus to reach the nearest subway station). If there are no nearby subway stations, we'll just impute the value for duration as an arbitrary 15min + longest time required to reach any nearby subway station in our entire dataset.

## Post Box

In [ ]:
commute_and_nearby_data['post_box']

- Honestly speaking, again from my own personal experience (hence, another assumption), it is very rare for one to use the post box nowadays when everything (settling utility bills, sending mail, ...) is done online. So we could just impute 1 (if any post box is present) or 0 (if no post boxes are present) as our feature to our final dataset.

## ATM

In [ ]:
commute_and_nearby_data['atm']

- Again, from my own personal experience (hence, another assumption), atms are very accessible around Singapore (in Shopping malls and in subway stations). So again, we could just impute 1 (if any atm is present) or 0 (if no atms are present) as our feature to our final dataset. Though, I expect this to be a really unimportant feature, especially when most things are paid through debit / credit cards nowadays.

## Park

In [ ]:
commute_and_nearby_data['park']

- Seems like the property we chose does not have a park nearby. However, to simplify things, I'm making another assumption yet again that having a park nearby isn't the most important feature to consider when determing house prices. Hence, we'll again impute 1 (if any park is present) or 0 (if no parks are present) as our feature to our final dataset.

## Supermarket

In [ ]:
commute_and_nearby_data['supermarket']

- In contrast to the previous 3, having a supermarket nearby a property might be an important feature that is accounted for in its price, especially for families that use public transport such as the subway (mrt) and buses frequently. Hence, I do expect to see a high correlation between the number of supermarkets accessible and the number of subway stations accessible. We'll include the number of supermarkets accessible as well as the average `duration` for transit.

## Clinic

In [ ]:
commute_and_nearby_data['clinic']

- Since healthcare accessibility is so important nowadays, this is likely another important feature that is considered, especially with the growing ageing population in Singapore. Again, we will include the number of clinics accessible and the `duration` by walking.

## Bus Station

In [ ]:
commute_and_nearby_data['bus_station']

- Similar to subway stations, public transport is especially important for the average, non-crazy-rich Singaporean. I'll expect this feature to be somewhat negatively correlated with house prices later on in our analysis along with the number of subway stations. If one is crazy-rich, it'll likely mean that they'll be using more private transportation modes rather than take the subway or bus. Again, we will take the total number of bus stations nearby and the average walking `duration`.

## Post Office

In [ ]:
commute_and_nearby_data['post_office']

- Like `post_box`, `atm`, and `park`, the number of post offices nearby aren't really an important feature to consider when determining the price of the house because of how rarely it's used nowadays. Similarly, we will impute 1 (if any post office is present) or 0 (if no post offices are present) as our feature to our final dataset.

## Banks

In [ ]:
commute_and_nearby_data['bank']

- The banks accessible to a house is kind of similar to the ATMs accessible in a way, but definitely a more important feature in my opinion since there're so many more services provided by banks than just plain ATMs. 
- In other countries, having an ATM or Bank nearby could signal that the neighbourhood might be "safer" and hence drive up the price of the real estate. However, in Singapore, where everywhere is safe, having a bank nearby does not really signal anything much other than convienience to deposit cash and possibly large withdrawals. I'll choose to include the number of banks nearby as well as the average transit `duration` to our final dataset.

## Commute

In [ ]:
commute_and_nearby_data['commute']

- This feature gives us a gauge of how long and how much it'll cost to get to 3 high traffic areas in Singapore, namely Changi Airport, Raffles Place (our Central Business District), and Orchard Road (our primary Shopping District). In my opinion, this is a pretty important feature as it gives us a good proxy as to how accessible Singapore in general is to the real estate in question. 
- I would like to include information about each of these places into our dataset as I believe that different demographics would care about the accessbility to these locations differently. For example, the wealthier someone is in Singapore, the more likely that they might care less about how far Changi Airport is and more of how close Orchard Road is, since it's also where most of the luxury brands are located at. Hence, I'll include the transit `duration` to each of these locations in the final dataset. 
- I'll expect there to be a generally negative correlation between the `duration` to Changi Airport and house price, and a strongly positive correlation between `duration` to Orchard Road and house price.

## Property Features

In [ ]:
# Let's see the property features
# of first property in our list
with open(join(properties[0], 'property_feats.json'), 'r') as f:
    property_feats = json.load(f)
    
property_feats

- `property_name`: The property name is by far the most important feature in our entire dataset because everyone obviously prices a house solely based on the name, HAHA. But truthfully speaking, it is logical that the name of the property might encode some information about how luxurious the property is and hence the price. However, in my opinion, it's not really significant enough for us to go through the trouble of performing sentiment analysis on the name of each property to give it a score on how luxurious the name sounds so we'll exclude it from our final dataset.
- `nearest_mrt`: We've already accounted for this in the `subway_station` feature above.
- `location`: All we really need here is the "District" number, which can be found in `key_details` so we'll ignore  this.
- `summary_feats`: This feature is pretty important, we'll want to keep `num_beds`, `num_baths`, `sqft`, `psf` / `price` (Our endogenous variable later for regression).
- `key_details`: Again, an important feature. We'll want to keep the categorical variable `Facing` and also keep the `District` variable later so we can compare the clusters we've found with these districts.
- `ammenities`: For this feature, we'll just keep the number of ammenities available as a feature.
- `project_overview`: We'll keep everything from this except for the `name` as they could all be beneficial in prediction
    - `total units`: Properties with more units are likely not as luxurious as those with very few and hence cannot command a high price
    - `year of completion`: Newer properties are likely to be more expensive than older ones
    - `tenure`: In some cases when the 2 properties being compared have very similar attributes, whether a property is [Freehold VS Leasehold](https://www.99.co/blog/singapore/freehold-vs-leasehold-condos/) could have a large effect on the sale price.
    - `link`: I just want to keep this so it's easier for us to get the transaction history of the clusters we're interested in performing regression analaysis after the cluster analysis section of the project.

<a id='transform'></a>

---
# Transform Part 1

We'll now transform our data.

## Formatting and Cleaning 

In [ ]:
# Import all the formatting and cleaning functions
# that we've created
import clean_99co
from clean_99co import *

reload(clean_99co);

In [ ]:
# Clean Schools
get_schools(commute_and_nearby_data)

In [ ]:
# Clean Subway Stations / MRTs
get_subway_stations(commute_and_nearby_data)

In [ ]:
# Clean Post Box
get_post_boxes(commute_and_nearby_data)

In [ ]:
# Clean ATM
get_atms(commute_and_nearby_data)

In [ ]:
# Clean Park
get_parks(commute_and_nearby_data)

In [ ]:
# Clean Supermarkets
get_supermarkets(commute_and_nearby_data)

In [ ]:
# Clean Clinics
get_clinics(commute_and_nearby_data)

In [ ]:
# Clean Bus Stations
get_bus_stations(commute_and_nearby_data)

In [ ]:
# Clean Post Office
get_post_offices(commute_and_nearby_data)

In [ ]:
# Clean Commute
get_commute(commute_and_nearby_data)

In [ ]:
# Clean Property Features
get_cleaned_features(property_feats)

Now that we've had a closer look about how our cleaned data will look, let's run these functions through each property we have stored in our dataset and create our finalized dataset.

In [ ]:
# Clean each property's commute_and_nearby_data.json
# and property_feats.json
# def clean_all_property_data(properties):
cleaned_property_df = pd.DataFrame()
for idx in tqdm(range(len(properties))):
    if isfile(join(properties[idx], 'commute_and_nearby_data.json')) and isfile(join(properties[idx], 'property_feats.json')):
        with open(join(properties[idx], 'commute_and_nearby_data.json'), 'r') as f1:
            commute_and_nearby_data = json.load(f1)
        with open(join(properties[idx], 'property_feats.json'), 'r') as f2:
            property_feats = json.load(f2)
        cleaned_property_df = pd.concat([
            cleaned_property_df,
            pd.DataFrame({
                **get_schools(commute_and_nearby_data),
                **get_subway_stations(commute_and_nearby_data),
                **get_post_boxes(commute_and_nearby_data),
                **get_atms(commute_and_nearby_data),
                **get_parks(commute_and_nearby_data),
                **get_supermarkets(commute_and_nearby_data),
                **get_clinics(commute_and_nearby_data),
                **get_bus_stations(commute_and_nearby_data),
                **get_post_offices(commute_and_nearby_data),
                **get_commute(commute_and_nearby_data),
                **get_cleaned_features(property_feats)
            }, index=[idx])
        ], axis=0, sort=True)

In [ ]:
cleaned_property_df.head(10)

In [ ]:
cleaned_property_df.columns

Seems like we've still quite abit of missing values that we need to deal with, let's save this dataframe first to `.csv` so we can use it later.

In [ ]:
cleaned_property_df.to_csv('./data/99.co/raw_final_dataset_{}.csv'.format(date.today().strftime("%Y_%m_%d")))

<a id='resources'></a>

---
## Resources
- https://www.99.co/blog/singapore/a-general-guide-to-types-of-housing-in-singapore/
- https://www.99.co/blog/singapore/freehold-vs-leasehold-condos/